# 初始化

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

from dotenv import load_dotenv
load_dotenv(dotenv_path='', override=True)

from langchain_deepseek import ChatDeepSeek
llm = ChatDeepSeek(model='deepseek-chat', api_key=os.getenv('DEEPSEEK_API_KEY'))

# 添加工具

集成一个网络搜索工具，聊天机器人可以利用这个工具来与外界交互，从而查找相关信息并提供更好的回复。

In [ ]:
from typing import Annotated

from langchain_tavily import TavilySearch
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


In [ ]:
tool = TavilySearch(max_results=2)

In [ ]:
tool.invoke("What's a 'node' in LangGraph?")

In [ ]:
# 定义状态，确定基本信息

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

In [ ]:
# 添加工具列表
llm_with_tools = llm.bind_tools([tool])

In [ ]:
def node(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


graph_builder.add_node("node", node)

In [ ]:
tool_node = ToolNode(tools=[tool])

graph_builder.add_node("tools", tool_node)

In [ ]:
graph_builder.add_edge(START, "node")

graph_builder.add_conditional_edges(
    "node",
    tools_condition,
)

# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "node")

In [ ]:
graph = graph_builder.compile()

In [ ]:
# 可视化

from IPython.display import Image, display
try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    print("displsy error")

In [ ]:
for event in graph.stream(
    {"messages": [{"role": "user", "content": "What do you know about LangGraph?"}]}
):
    for value in event.values():
        print("Assistant:", value["messages"][-1].content)

# 添加记忆

LangGraph 通过持久化检查点（persistent checkpointing）解决了这个问题。

如果在编译图时提供一个 checkpointer，并在调用图时提供一个 thread_id，LangGraph 会在每个步骤后自动保存状态。

当你再次使用相同的 thread_id 调用图时，图会加载其已保存的状态，从而让聊天机器人可以从上次中断的地方继续。

In [ ]:
from typing import Annotated

from langchain_tavily import TavilySearch
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

tool = TavilySearch(max_results=2)
tools = [tool]
llm_with_tools = llm.bind_tools(tools)


def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)


graph_builder.set_entry_point("chatbot")
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")


In [ ]:
# 将所有内容保存在内存中

memory = InMemorySaver()

graph = graph_builder.compile(checkpointer=memory)

In [ ]:
from IPython.display import Image, display
try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    print("displsy error")

In [ ]:
config = {"configurable": {"thread_id": "1"}}

user_input = "Hi there! My name is Will."

# The config is the **second positional argument** to stream() or invoke()!
events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)

for event in events:
    event["messages"][-1].pretty_print()

In [ ]:
user_input = "Remember my name?"

# The config is the **second positional argument** to stream() or invoke()!
events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

In [ ]:
# The only difference is we change the `thread_id` here to "2" instead of "1"
events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    {"configurable": {"thread_id": "2"}},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

In [ ]:
# 检查状态

graph.get_state(config)